In [2]:
# In[1]: imports
import sapien.core as sapien
from IPython.display import display

In [ ]:
# replace these imports with wherever your classes live
from my_envs import (
    MoveNearGoogleBakedTexInSceneEnv,
    NewSceneEnv,
)

In [3]:
import numpy as np
import gymnasium as gym
from transforms3d.euler import euler2quat
from sapien.core import Pose
from mani_skill2_real2sim.envs.sapien_env import BaseEnv

In [4]:
# List your two env IDs here
ENV_IDS = [
    "MoveNearGoogleBakedTexInScene-v0",
    "NewScene",        # ← replace with your actual new‐scene ID
]

# default “CLI” settings from your script
OBS_MODE     = None
REWARD_MODE  = None
CONTROL_MODE = "arm_pd_ee_delta_pose_align_interpolate_by_planner_gripper_pd_joint_target_delta_pos_interpolate_by_planner"
RENDER_MODE  = "cameras"
CAMERA_CFGS  = {"add_segmentation": False}

def make_reset_options(env_id):
    """
    Copy the same reset logic you had under `if names_in_env_id_fxn(["MoveNear"])`
    """
    if "MoveNear" in env_id:
        # match the hard‐coded real‐eval pose from your script
        init_rot_quat = (
            Pose(q=euler2quat(0, 0, -0.09))
            * Pose(q=[0, 0, 0, 1])
        ).q
        return dict(
            obj_init_options   = {"episode_id": 0}, 
            robot_init_options = {
                "init_xy":       [0.35, 0.21],
                "init_rot_quat": init_rot_quat,
            },
        )
    else:
        # no special reset options
        return {}

In [5]:
# Loop through envs, make & reset, then print actors
for env_id in ENV_IDS:
    print(f"\n=== Loading scene: {env_id} ===")
    env: BaseEnv = gym.make(
        env_id,
        obs_mode     = OBS_MODE,
        reward_mode  = REWARD_MODE,
        control_mode = CONTROL_MODE,
        render_mode  = RENDER_MODE,
        camera_cfgs  = CAMERA_CFGS,
    )

    obs, info = env.reset(options=make_reset_options(env_id))
    print("Reset info:", info)

    actors = env.get_actors()  # now uses your helper
    print(f"Found {len(actors)} actors:")
    for a in actors:
        print(" •", a)


=== Loading scene: MoveNearGoogleBakedTexInScene-v0 ===


/work/users/t/i/tis/mmai/.venv/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment MoveNearGoogleBakedTexInScene-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(
[2025-04-30 18:29:10.680] [svulkan2] [error] GLFW error: X11: Failed to open display 172.26.114.214:10.0
[2025-04-30 18:29:10.680] [svulkan2] [warning] Continue without GLFW.


Reset info: {'scene_name': None, 'scene_offset': None, 'scene_pose': None, 'scene_table_height': 0.87, 'urdf_version': '', 'rgb_overlay_path': None, 'rgb_overlay_cameras': [], 'rgb_overlay_mode': 'background', 'disable_bad_material': False, 'episode_model_ids': ('blue_plastic_bottle', 'baked_opened_pepsi_can', 'orange'), 'episode_model_scales': [1.0, 1.0, 1.0], 'episode_source_obj_name': 'blue_plastic_bottle', 'episode_target_obj_name': 'baked_opened_pepsi_can', 'episode_source_obj_init_pose_wrt_robot_base': Pose([0.664308, 0.225808, 0.895612], [0.51431, 0.529205, -0.457088, -0.496486]), 'episode_target_obj_init_pose_wrt_robot_base': Pose([0.688933, -0.0683567, 0.872771], [0.0318171, 0.0318166, -0.706391, -0.706391]), 'episode_id': 0}
Found 4 actors:
 • Actor(name="arena", id="38")
 • Actor(name="blue_plastic_bottle", id="39")
 • Actor(name="baked_opened_pepsi_can", id="40")
 • Actor(name="orange", id="41")

=== Loading scene: NewScene ===
Loading Overlay Image from: /work/users/t/i/ti

/work/users/t/i/tis/mmai/.venv/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_actors to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_actors` for environment variables or `env.get_wrapper_attr('get_actors')` that will search the reminding wrappers.
  logger.warn(
[2025-04-30 18:29:15.399] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.


Reset info: {'scene_name': 'google_pick_coke_can_1_v4', 'scene_offset': None, 'scene_pose': None, 'scene_table_height': 0.87, 'urdf_version': 'recolor_tabletop_visual_matching_1', 'rgb_overlay_path': '/work/users/t/i/tis/mmai/ManiSkill2_real2sim/data/real_inpainting/table_env1.png', 'rgb_overlay_cameras': ['overhead_camera'], 'rgb_overlay_mode': 'background', 'disable_bad_material': False, 'episode_model_ids': ('opened_coke_can', 'opened_redbull_can', 'apple'), 'episode_model_scales': [1.0, 1.0, 1.0], 'episode_source_obj_name': 'opened_redbull_can', 'episode_target_obj_name': 'apple', 'episode_source_obj_init_pose_wrt_robot_base': Pose([0.674559, 0.0799223, 0.874034], [0.0266965, 0.0368983, -0.700547, -0.712151]), 'episode_target_obj_init_pose_wrt_robot_base': Pose([0.489765, -0.0872634, 0.844358], [0.0443316, 0.0011375, 0.0133334, -0.998927]), 'episode_id': 33}
Found 4 actors:
 • Actor(name="arena", id="38")
 • Actor(name="opened_coke_can", id="39")
 • Actor(name="opened_redbull_can",

In [20]:
# list all visual sub-bodies / render shapes
print("Visual shapes in arena:")
for vb in actors[0].get_visual_bodies():
    print(vb)
    for rs in vb.get_render_shapes():
        mesh = rs.mesh  # correct property
        # if this mesh was loaded from a file, it may have a `.path`
        mesh_path = getattr(mesh, "path", None)
        if mesh_path:
            print(" •", mesh_path)
        else:
            # fallback to printing the mesh object itself
            print(" •", mesh)

from sapien.core import KinematicType, MeshGeometry
# 2) iterate its rigid bodies → collision shapes → MeshGeometry
mesh_files = set()
for rb in actor[0].get_rigid_bodies():
    for cs in rb.get_collision_shapes():
        geom = cs.geometry
        if isinstance(geom, MeshGeometry):
            mesh_files.add(geom.filename)

# 3) print out every mesh file that defines your arena
print("Mesh files in arena (floor, table, walls, etc):")
for fn in sorted(mesh_files):
    print(" •", fn)

Visual shapes in arena:
 • <sapien.core.pysapien.VulkanRenderMesh object at 0x7ff71da606b0>
 • <sapien.core.pysapien.VulkanRenderMesh object at 0x7ff71dbe6330>


ImportError: cannot import name 'KinematicType' from 'sapien.core' (/work/users/t/i/tis/mmai/.venv/lib/python3.11/site-packages/sapien/core/__init__.py)

In [21]:
from sapien.core import MeshGeometry

# assume actors[0] is your arena actor
arena = actors[0]

mesh_files = set()
for rb in arena.get_rigid_bodies():
    for cs in rb.get_collision_shapes():
        geom = cs.geometry
        if isinstance(geom, MeshGeometry):
            mesh_files.add(geom.filename)

print("Mesh files in arena (table, floor, walls, etc):")
for fn in sorted(mesh_files):
    print(" •", fn)

ImportError: cannot import name 'MeshGeometry' from 'sapien.core' (/work/users/t/i/tis/mmai/.venv/lib/python3.11/site-packages/sapien/core/__init__.py)